In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import sys
sys.path.append('../.')
from lib import get_data

In [2]:
df = get_data.get_model_data(date_range=(0,7), pred_day=7)
df.head()

,state,county,fips,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,...,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,workplaces_percent_change_from_baseline,transit_stations_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018,pop_2018,day_7_delta_cases,day_7_delta_deaths
0,alabama,autauga,1001,11.500,0.375,0.250,31.250,81.625000,48.762500,52.875000,...,-31.500,16.50,-38.000000,NaN,9.875,NaN,41618.0,55601.0,-20,1
1,alabama,baldwin,1003,20.875,0.500,0.000,31.250,86.625000,47.662500,53.725000,...,-42.125,12.25,-31.875000,-24.5,-12.500,-33.5,45596.0,218022.0,17,0
2,alabama,barbour,1005,12.625,0.000,0.000,24.000,66.583333,41.627083,49.241667,...,NaN,NaN,-31.000000,NaN,NaN,NaN,35199.0,24881.0,-11,-1
3,alabama,bibb,1007,16.000,0.000,0.000,26.375,60.375000,36.375000,44.337500,...,NaN,NaN,-32.333333,NaN,NaN,NaN,30254.0,22400.0,6,0
4,alabama,blount,1009,10.625,0.000,0.075,30.950,89.750000,41.680000,47.140000,...,-29.250,16.80,-34.750000,NaN,0.000,NaN,34976.0,57840.0,-12,0


In [3]:
df = df._get_numeric_data().drop(['fips'],axis=1).dropna()
df.head()

,cases,deaths,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,...,retail_and_recreation_percent_change_from_baseline,residential_percent_change_from_baseline,workplaces_percent_change_from_baseline,transit_stations_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,income_2018,pop_2018,day_7_delta_cases,day_7_delta_deaths
1,20.875,0.500,0.0000,31.250000,86.6250,47.662500,53.725000,59.63750,56.025000,67.625000,...,-42.125,12.250000,-31.875,-24.500,-12.500,-33.500000,45596.0,218022.0,17,0
14,21.000,1.000,0.0625,31.833333,89.0625,43.135417,48.777083,54.99375,51.904167,62.291667,...,-37.250,14.142857,-34.625,-30.250,4.500,41.714286,39607.0,83442.0,12,1
16,14.125,0.000,0.2500,30.875000,91.8750,40.800000,47.112500,53.85000,48.525000,60.212500,...,-37.625,14.000000,-31.000,-37.000,-4.125,-4.000000,31916.0,71385.0,-3,-2
18,29.875,2.625,0.0000,24.125000,71.7500,46.350000,51.062500,55.22500,49.175000,62.375000,...,-38.000,13.000000,-35.625,-31.625,-3.875,-23.000000,36918.0,102501.0,30,4
22,23.500,1.000,0.0000,33.000000,96.0000,50.900000,56.825000,62.66250,59.712500,70.425000,...,-42.250,13.625000,-34.500,-31.875,-9.125,3.500000,42398.0,104722.0,17,1


In [5]:
X = df.drop(['day_7_delta_cases','day_7_delta_deaths'], axis = 1)
y = df.day_7_delta_cases

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
rf = RandomForestRegressor(max_features='auto', oob_score=True, random_state=1, n_jobs=-1)
param_grid = { "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 8, 12], "n_estimators": [10, 50, 100, 200, 500]}
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
gs = gs.fit(X_train, y_train)

gs.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=-1, oob_score=True,
                      random_state=1, verbose=0, warm_start=False)

In [19]:
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=-1, oob_score=True,
                      random_state=1, verbose=0, warm_start=False)

rf.fit(X_train, y_train)
print("%.4f" % rf.oob_score_)

0.4238


In [21]:
pd.concat((pd.DataFrame(X_train.columns, columns = ['variables']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:10]


,variables,importance
0,cases,0.682877
60,transit_stations_percent_change_from_baseline,0.024776
62,parks_percent_change_from_baseline,0.024310
58,residential_percent_change_from_baseline,0.018227
45,windSpdMin,0.017914
63,income_2018,0.016531
57,retail_and_recreation_percent_change_from_base...,0.014930
44,windDir100mAvg,0.012362
61,grocery_and_pharmacy_percent_change_from_baseline,0.010089
43,windDir80mAvg,0.009038
